In [194]:
#import all the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import preprocessing
%pylab inline

# To ignore some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

Populating the interactive namespace from numpy and matplotlib


In [195]:
# Load ACS and garden data
ACS = pd.read_csv("../data/ACS_Merge_Norm_Final.csv",index_col=0)
# Load crime and garden data
crime = pd.read_csv("../data/Crime_totals_normalized.csv",index_col=0)

In [196]:
ACS.head()

,GeoType,NTA,GeoID,Borough,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
0,NTA2010,Bath Beach,BK27,Brooklyn,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
1,NTA2010,Bay Ridge,BK31,Brooklyn,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
2,NTA2010,Bedford,BK75,Brooklyn,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
3,NTA2010,Bensonhurst East,BK29,Brooklyn,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
4,NTA2010,Bensonhurst West,BK28,Brooklyn,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [197]:
type(ACS)

pandas.core.frame.DataFrame

In [198]:
ACS.columns

Index(['GeoType', 'NTA', 'GeoID', 'Borough', 'Pop_1E', 'MaleEN', 'FemEN',
       'PopUnd18N', 'PopOv65N', 'HispN', 'NotHispN', 'WhiteN', 'BlackN',
       'NativeN', 'AsianN', 'PacificN', 'OtherRace1N', 'OtherRace2N', 'HHPop',
       'HHINC_U10N', 'HHINC_10t14N', 'HHINC_15t24N', 'HHINC_25t34N',
       'HHINC_35t49N', 'HHINC_50t74N', 'HHINC_75t99N', 'HHINC_100t149N',
       'HHINC_150t199N', 'HHINC_200plN', 'Pop16yrsOver', 'Labor_forceN',
       'CivilianLF', 'CLF_UnemployedN', 'PopOver25', 'EduOtherN',
       'EduBachelor', 'EduGradProf', 'GardenCounts', 'HasGarden'],
      dtype='object')

In [199]:
ACS = ACS.set_index('NTA')
ACS.head()

,GeoType,GeoID,Borough,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,NTA2010,BK27,Brooklyn,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
Bay Ridge,NTA2010,BK31,Brooklyn,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
Bedford,NTA2010,BK75,Brooklyn,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
Bensonhurst East,NTA2010,BK29,Brooklyn,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
Bensonhurst West,NTA2010,BK28,Brooklyn,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [200]:
ACS = ACS.drop(columns=['GeoID','GeoType','Borough'])
ACS.head()

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,0.436933,0.012182,0.000332,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
Bay Ridge,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,0.634329,0.019094,0.002287,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
Bedford,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,0.350607,0.405826,0.000876,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
Bensonhurst East,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,0.465153,0.008262,0.000000,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
Bensonhurst West,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,0.426773,0.008740,0.001211,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [201]:
crime.head()

,FELONY,MISDEMEANOR,VIOLATION,TOTAL_CRIME,Population,GardenCounts,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
ntaname,,,,,,,,,,
Allerton-Pelham Gardens,752.0,1925.0,444.0,3121.0,32450.0,1.0,0.023174,0.059322,0.013683,0.096179
Annadale-Huguenot-Prince's Bay-Eltingville,431.0,1215.0,464.0,2110.0,28698.0,0.0,0.015018,0.042337,0.016168,0.073524
Arden Heights,263.0,785.0,324.0,1372.0,26482.0,0.0,0.009931,0.029643,0.012235,0.051809
Astoria,4002.0,8082.0,2211.0,14295.0,77252.0,1.0,0.051804,0.104619,0.028621,0.185044
Auburndale,557.0,713.0,258.0,1528.0,20609.0,0.0,0.027027,0.034597,0.012519,0.074142


In [202]:
# merge crime and ACS data
data = ACS.merge(crime[['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']], how='right', left_index=True, right_index = True)
data.head()

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
ntaname,,,,,,,,,,,,,,,,,,,,,
Allerton-Pelham Gardens,32450.0,0.469492,0.530508,0.210293,0.174391,0.341941,0.658059,0.221202,0.328475,0.000000,...,22699.0,0.741619,0.150051,0.108331,1.0,1.0,0.023174,0.059322,0.013683,0.096179
Annadale-Huguenot-Prince's Bay-Eltingville,28698.0,0.490243,0.509757,0.206530,0.172416,0.070562,0.929438,0.886682,0.005053,0.001045,...,20340.0,0.662881,0.208014,0.129105,0.0,0.0,0.015018,0.042337,0.016168,0.073524
Arden Heights,26482.0,0.493996,0.506004,0.223812,0.137716,0.126614,0.873386,0.779473,0.014878,0.000529,...,18338.0,0.663158,0.219271,0.117570,0.0,0.0,0.009931,0.029643,0.012235,0.051809
Astoria,77252.0,0.486563,0.513437,0.126637,0.127505,0.269080,0.730920,0.499184,0.037151,0.000634,...,61188.0,0.529107,0.300075,0.170818,1.0,1.0,0.051804,0.104619,0.028621,0.185044
Auburndale,20609.0,0.494007,0.505993,0.185938,0.169538,0.132563,0.867437,0.367898,0.010384,0.001601,...,15457.0,0.629747,0.212590,0.157663,0.0,0.0,0.027027,0.034597,0.012519,0.074142


In [203]:
data.shape

(192, 39)

In [204]:
#Y = data[['GardenCounts', 'HasGarden']]
#Y.head()

In [205]:
#X = data.drop(columns=['GardenCounts', 'HasGarden'])
#X.shape

In [206]:
#from sklearn.model_selection import train_test_split 
#X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.8, test_size = 0.2, random_state = 42)

In [207]:
# split into train, test, and validate data
train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [208]:
print(train.shape)
print(validate.shape)
print(test.shape)

(115, 39)
(38, 39)
(39, 39)


In [209]:
# extract garden target columns from data 
Ytrain = pd.DataFrame(train[['GardenCounts', 'HasGarden']])
Xtrain = pd.DataFrame(train.drop(columns=['GardenCounts', 'HasGarden']))
Yvalidate = pd.DataFrame(validate[['GardenCounts', 'HasGarden']])
Xvalidate = pd.DataFrame(validate.drop(columns=['GardenCounts', 'HasGarden']))
Ytest = pd.DataFrame(test[['GardenCounts', 'HasGarden']])
Xtest = pd.DataFrame(test.drop(columns=['GardenCounts', 'HasGarden']))

In [210]:
# scale X data
scaler = preprocessing.StandardScaler().fit(Xtrain)
XtrainS = pd.DataFrame(scaler.transform(Xtrain))
XvalidateS = pd.DataFrame(scaler.transform(Xvalidate))
XtestS = pd.DataFrame(scaler.transform(Xtest))

## Logistic Model with all features

In [211]:
# problem: there are 21 NaN values, distributed over the XtrainS, XtestS, and XvalidateS dataframes differently depending on the train, test, validate split at the beginning. The NaNs existed before scaling. 
XtrainS.isnull().values.sum()

0

In [212]:
XvalidateS.isnull().values.sum()

11

In [213]:
XtestS.isnull().values.sum()

10

In [214]:
# don't know if this is problematic, but did fillna to resolve the error
XtrainS = XtrainS.fillna(0);
XvalidateS = XvalidateS.fillna(0);
XtestS = XtestS.fillna(0);

In [215]:
# Learn logistic regression over the training sample
logit = LogisticRegression(C=10000)
logit = logit.fit(XtrainS, Ytrain['HasGarden']) #fit logistic regression model to training data

In [216]:
# create garden predictions over test sample
gardenPred = logit.predict(XtestS)
gardenPred.shape

(39,)

In [217]:
# calculate out of sample accuracy
test_accuracy = logit.score(XtestS,Ytest['HasGarden'])
test_accuracy

0.717948717948718

### Logistic Model with Sparse PCA (try with n=10 principle components)

In [218]:
from sklearn.decomposition import SparsePCA
#from sklearn.decomposition import PCA

In [219]:
# Was curious if switching to arrays would solve the error
#XtrainS_arr = XtrainS.to_numpy()
#XvalidateS_arr = XvalidateS.to_numpy()
#XtestS_arr = XtestS.to_numpy()

In [220]:
# apply sparse PCA
spca = SparsePCA(XtrainS.shape[1]) # doing .shape[1] solved an error but I don't know why
XtrainP = spca.fit_transform(XtrainS)
XvalidateP = spca.transform(XvalidateS)
XtestP = spca.transform(XtestS)

/Users/cmvandev/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/sparse_pca.py:170: DeprecationWarning: normalize_components=False is a backward-compatible setting that implements a non-standard definition of sparse PCA. This compatibility mode will be removed in 0.22.
  DeprecationWarning)


In [221]:
print(XtrainP.shape)
print(XvalidateP.shape)
print(XtestP.shape)

(115, 37)
(38, 37)
(39, 37)


In [222]:
# Learn logistic regression over the first 5 principle components of sparse PCA training sample
logit_spca = LogisticRegression(C=10000)
logit_spca = logit_spca.fit(XtrainP[:,:10], Ytrain['HasGarden']) #fit logistic regression model to training data

In [223]:
# create binary garden predictions over test sample
labelPred_spca = logit_spca.predict(XtestP[:,:10])
print(labelPred_spca)

[1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1.]


In [224]:
# compute out of sample accuracy
test_accuracy_spca= logit_spca.score(XtestP[:,:10],Ytest['HasGarden'])
test_accuracy_spca 

0.717948717948718

### Select optimal number of principle components for classification

In [225]:
# select optimal number of principle components for the classification in terms of the out-of-sample accuracy measured over the validation sample
optimal_SPCA = np.zeros(shape=(37,2))
for n in range(1,37):
    #Learn logistic regression over the training sample
    logit_spca_opt = LogisticRegression(C=10000)
    logit_spca_opt = logit_spca_opt.fit(XtrainP[:,:n], Ytrain['HasGarden']) #fit logistic regression model to training data
    # compute out of sample accuracy over validation set
    validation_accuracy = logit_spca_opt.score(XvalidateP[:,:n],Yvalidate['HasGarden'])
    #print(n, validation_accuracy)
    optimal_SPCA[n,0] = n
    optimal_SPCA[n,1] = validation_accuracy

In [226]:
optimal_SPCA_df = pd.DataFrame(optimal_SPCA, index = optimal_SPCA[:,0])
optimal_SPCA_df.columns = ['n', 'accuracy']
optimal_SPCA_df.sort_values(by=['accuracy'], ascending = False).head(1)

,n,accuracy
4.0,4.0,0.736842


In [227]:
#Out of sample (test) accuracy for n = 6
logit_n6 = LogisticRegression(C=10000)
logit_n6 = logit_n6.fit(XtrainP[:,:6], Ytrain['HasGarden'])
out_of_sample_accuracy_test_n6= logit_n6.score(XtestP[:,:6],Ytest['HasGarden'])
out_of_sample_accuracy_test_n6
#The out of sample accuracy over the test set for 6 principal components is 86%

0.6923076923076923

### Logistic Model with Regularization